In [2]:
!pip install bs4

    100% |################################| 112kB 2.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import requests
import urllib.request
import datetime
from bs4 import BeautifulSoup
import pandas as pd


# Ontario

Since Ontario shows the most recent news on the first page, the range
will need to continue to be expanded to capture all posts. Generally ~4 pages capture ~2 weeks.

In [53]:
targets = ['https://news.ontario.ca/archive/en?page=' + str(i) for i in range(1,4)]

# Specific structure for news.contario.ca/archive
links = []
for target in targets:
    if target.startswith('https://news.ontario.ca/archive/'):
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.findAll('article')
        
        for article in articles:
            smallersoup = BeautifulSoup(str(article), "html.parser")
            link = smallersoup.findAll('a')[0]['href']
            title = smallersoup.findAll('a')[0].string
            pub_date = smallersoup.time['datetime']
            
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            full_text = linksoup.article.text
            
            row = [pub_date, link, title, full_text]
            links.append(row)
            
            # Get this link and copy full text


In [54]:
df = pd.DataFrame(links, columns=['start_date', 'source_url', 'source_title', 'source_full_text'])
df

,start_date,source_url,source_title,source_full_text
0,2020-05-02,http://news.ontario.ca/opo/en/2020/05/ontario-...,Ontario Ready to Respond to Spring Flooding Se...,\nOntario Ready to Respond to Spring Flooding ...
1,2020-05-01,http://news.ontario.ca/mndmf/en/2020/05/ontari...,Ontario Providing Support for Industrial and C...,\nOntario Providing Support for Industrial and...
2,2020-05-01,http://news.ontario.ca/opo/en/2020/05/ontario-...,Ontario Takes Additional Steps to Improve Heal...,\nOntario Takes Additional Steps to Improve He...
3,2020-05-01,http://news.ontario.ca/opo/en/2020/05/certain-...,Certain Businesses Allowed to Reopen Under Str...,\nCertain Businesses Allowed to Reopen Under S...
4,2020-04-30,http://news.ontario.ca/mto/en/2020/04/governme...,Government Launches New Tool to Support Trucki...,\nGovernment Launches New Tool to Support Truc...
5,2020-04-30,http://news.ontario.ca/mnr/en/2020/04/11600-in...,"$11,600 in Fines for Illegal Migratory Bird Hunt","\n$11,600 in Fines for Illegal Migratory Bird ..."
6,2020-04-30,http://news.ontario.ca/opo/en/2020/04/ontario-...,Ontario Providing Employers with Workplace Saf...,\nOntario Providing Employers with Workplace S...
7,2020-04-29,http://news.ontario.ca/opo/en/2020/04/more-fro...,More Frontline Workers Eligible for Emergency ...,\nMore Frontline Workers Eligible for Emergenc...
8,2020-04-28,http://news.ontario.ca/edu/en/2020/04/minister...,Minister Appoints Investigator to Examine Peel...,\nMinister Appoints Investigator to Examine Pe...
9,2020-04-28,http://news.ontario.ca/mnr/en/2020/04/seeking-...,Seeking Public's Help with Discarded Deer Part...,\nSeeking Public's Help with Discarded Deer Pa...


In [55]:
df.to_csv('ontario-sources-apr10_may2.csv')

In [56]:
df.shape

(45, 4)

# Manitoba

Retrieve all news releases in 2020 for the Province of Manitoba.

In [44]:
url_base = 'https://news.gov.mb.ca'
targets = [url_base + '/news/index.html?month=' + str(i) + '&year=2020&day=01&bgnG=GO&d=' for i in range(1,12)]


In [46]:
%time

links = []
for target in targets:
    if target.startswith(url_base): #manitoba
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.findAll("div", {"class": "maincontent"})
        smallersoup = BeautifulSoup(str(items), "html.parser")
        for article in smallersoup.findAll('h2'):
            a = article.a
            relative_link = a['href']
            link = url_base + relative_link.split('..')[-1]
            title = a.string
        
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            
            date_text = linksoup.findAll("span", {"class": "article_date"})[0].string
            date = pd.to_datetime(date_text, format='%B %d, %Y')
            pub_date = date.strftime('%m/%d/%Y')
            
            full_text = linksoup.findAll("div", {"class": ""})[0].text

            
            row = [pub_date, link, title, full_text]
            links.append(row)
            
            # Get this link and copy full text

In [47]:
df = pd.DataFrame(links, columns=['start_date', 'source_url', 'source_title', 'source_full_text'])
df.to_csv('manitoba-sources.csv')
df.shape

(260, 4)